In [1]:
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import MetaPath2Vec
from torch_sparse import transpose
import torch

/zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


OSError: /zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev

In [32]:
dataset = PygNodePropPredDataset(name = "ogbn-mag")

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

# train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
# valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
# test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

data = dataset[0]

/zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  se

In [33]:
# metapath = [
#     ('author', 'writes', 'paper'),
#     ('paper', 'cites', 'paper'),
#     ('paper', 'has_topic', 'field_of_study'),
#     ('author', 'affiliated_with', 'institution'),
# ]

data.edge_index_dict[('institution', 'employs', 'author')] = torch.transpose(
    data.edge_index_dict[('author', 'affiliated_with', 'institution')],
    None, m=data.num_nodes_dict['author'],
    n=data.num_nodes_dict['institution'])[0]

data.edge_index_dict[('paper', 'written_by', 'author')] = torch.transpose(
    data.edge_index_dict[('author', 'writes', 'paper')], None,
    m=data.num_nodes_dict['author'], n=data.num_nodes_dict['paper'])[0]

data.edge_index_dict[('field_of_study', 'contains', 'paper')] = torch.transpose(
    data.edge_index_dict[('paper', 'has_topic', 'field_of_study')], None,
    m=data.num_nodes_dict['paper'],
    n=data.num_nodes_dict['field_of_study'])[0]

print(data)

metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'has_topic', 'field_of_study'),
    ('field_of_study', 'contains', 'paper'),
    ('paper', 'written_by', 'author'),
    ('author', 'affiliated_with', 'institution'),
    ('institution', 'employs', 'author'),
    ('author', 'writes', 'paper'),
    ('paper', 'cites', 'paper'),
    ('paper', 'written_by', 'author'),
]

if torch.cuda.is_available():
    device = torch.device('cuda')

model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=20, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)


loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Loss: {total_loss / log_steps:.4f}')
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Acc: {acc:.4f}')


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('paper', batch=torch.arange(0, 736389).to(device))
    y = data.y_dict['paper'].ravel()

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)


for epoch in range(1, 6):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

TypeError: transpose() received an invalid combination of arguments - got (Tensor, NoneType, n=int, m=int), but expected one of:
 * (Tensor input, int dim0, int dim1)
 * (Tensor input, name dim0, name dim1)
